Import required packages


In [ ]:
import numpy as np
import pandas as pd 
import itertools

Warehouse Settings

In [ ]:
#2x2 Layout
warehouseSize=4

Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data=pd.read_csv('drive/MyDrive/SAKI4/warehousetraining2x2.txt', header=None)
train_data.columns = ['training']

Define States


In [ ]:
def define_states():
    content = ['0','r','b','w'] #0 = empty, r = red, b = blue, w = white
    commands = ['sr', 'sb', 'sw', 'rr', 'rb', 'rw'] #sb/sr/sw = store_blue/red/white, rb/rr/rw = restore blue/red/white

    states = []

    for x1 in content:
        for x2 in content:
            for x3 in content:
                for x4 in content:
                    for command in commands:
                        state = (x1, x2, x3, x4, command)
                        states.append(state)
    return states

states = define_states()

In [ ]:
states=define_states(4)
print(states)
print(len(states))

[('0', '0', '0', '0', 'sr'), ('0', '0', '0', 'r', 'sr'), ('0', '0', '0', 'b', 'sr'), ('0', '0', '0', 'w', 'sr'), ('0', '0', 'r', '0', 'sr'), ('0', '0', 'r', 'r', 'sr'), ('0', '0', 'r', 'b', 'sr'), ('0', '0', 'r', 'w', 'sr'), ('0', '0', 'b', '0', 'sr'), ('0', '0', 'b', 'r', 'sr'), ('0', '0', 'b', 'b', 'sr'), ('0', '0', 'b', 'w', 'sr'), ('0', '0', 'w', '0', 'sr'), ('0', '0', 'w', 'r', 'sr'), ('0', '0', 'w', 'b', 'sr'), ('0', '0', 'w', 'w', 'sr'), ('0', 'r', '0', '0', 'sr'), ('0', 'r', '0', 'r', 'sr'), ('0', 'r', '0', 'b', 'sr'), ('0', 'r', '0', 'w', 'sr'), ('0', 'r', 'r', '0', 'sr'), ('0', 'r', 'r', 'r', 'sr'), ('0', 'r', 'r', 'b', 'sr'), ('0', 'r', 'r', 'w', 'sr'), ('0', 'r', 'b', '0', 'sr'), ('0', 'r', 'b', 'r', 'sr'), ('0', 'r', 'b', 'b', 'sr'), ('0', 'r', 'b', 'w', 'sr'), ('0', 'r', 'w', '0', 'sr'), ('0', 'r', 'w', 'r', 'sr'), ('0', 'r', 'w', 'b', 'sr'), ('0', 'r', 'w', 'w', 'sr'), ('0', 'b', '0', '0', 'sr'), ('0', 'b', '0', 'r', 'sr'), ('0', 'b', '0', 'b', 'sr'), ('0', 'b', '0', 'w'

Calculate probabilities from training set for the creation of TPMs

In [ ]:
def calculate_probability():
  probabilities = (train_data.training.value_counts() / train_data.shape[0])
  #probabilities=probabilities.round(3)
  return probabilities


In [ ]:
 probabilities=calculate_probability()
 print(probabilities)
 #check if the sum of the probabilities is equal to 1
 print(1.0==sum(probabilities))
 

restore\tred      0.252415
store\tred        0.252415
store\twhite      0.125963
restore\twhite    0.125841
store\tblue       0.121683
restore\tblue     0.121683
Name: training, dtype: float64
True


**Generate TPMs**

for each action (position in the warehouse) create a TPM

In [ ]:
def generate_TPMs():
    TPMs = []

    for i in range(4):
        TPMs.append(np.zeros((4 ** warehouseSize * 6, 4 ** warehouseSize * 6), dtype=np.float16))

    probas={'sr':0.252415,'rr':0.252415,'sw':0.125963,'rw': 0.125841,'sb': 0.121683,'rb': 0.121683}   
   


    for i ,tpm in enumerate(TPMs):
        for row, state_0 in enumerate(states):
            for col, state_1 in enumerate(states):
                # store
                if state_0[-1][0] == 's':
                    # store possible
                    if state_0[i] == '0':
                        if state_0[:i] + (state_0[-1][1],) + state_0[i + 1:-1] == state_1[:-1]:
                            tpm[row][col] = probas[state_1[-1]]
                        continue
                    # store impossible
                    if state_0[:-1] == state_1[:-1]:
                        tpm[row][col] = probas[state_1[-1]]
                        continue
                # restore
                if state_0[-1][0] == 'r':
                    if state_0[-1][1] == 'r':
                        # restore possible
                        if state_0[i] == 'r':
                            if state_0[:i] + ('0',) + state_0[i + 1:-1] == state_1[:-1]:
                                tpm[row][col] = probas[state_1[-1]]
                            continue
                        # restore impossible: stay in state
                        if state_0[:-1] == state_1[:-1]:
                            tpm[row][col] = probas[state_1[-1]]
                            continue
                    if state_0[-1][1] == 'w':
                        # restore possible
                        if state_0[i] == 'w':
                            if state_0[:i] + ('0',) + state_0[i + 1:-1] == state_1[:-1]:
                                tpm[row][col] = probas[state_1[-1]]
                            continue
                        # restore impossible: stay in state
                        if state_0[:-1] == state_1[:-1]:
                            tpm[row][col] = probas[state_1[-1]]
                            continue
                    if state_0[-1][1] == 'b':
                        # restore possible
                        if state_0[i] == 'b':
                            if state_0[:i] + ('0',) + state_0[i + 1:-1] == state_1[:-1]:
                                tpm[row][col] = probas[state_1[-1]]
                            continue
                        # restore impossible: stay in state
                        if state_0[:-1] == state_1[:-1]:
                            tpm[row][col] = probas[state_1[-1]]
                            continue
    
    return TPMs

TPMs = generate_TPMs()
print(TPMs)

[array([[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.2524, 0.1217, 0.1259],
       [0.    , 0.    , 0.    , ..., 0.2524, 0.1217, 0.1259],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],
      dtype=float16), array([[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.2524, 0.1217, 0.1259],
       [0.    , 0.    , 0.    , ..., 0.2524, 0.1217, 0.1259],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],
      dtype=float16), array([[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],

In [ ]:
def generate_reward_matrix():
  reward_matrix=np.zeros((len(states), warehouseSize))
  #define reward values 
  #if the store and restore are possible the reward value depends on the Manhattan distance 
  #if the store and restore are not possible the reward is -100
  reward_values={0:-1,1:-2,2:-2,3:-3,4:-3,5:-4,"err":-100}
  for i, state in enumerate(states):
        # store command
        if state[-1][0] == 's':
            for pos in range(warehouseSize):
                #The store is possible when the field is empty
                if state[pos]=='0':
                  reward_matrix[i][pos] = reward_values[pos] 
                else : 
                 reward_matrix[i][pos] = reward_values['err'] 

        # restore command
        else:
            for pos in range(warehouseSize):
                #the restore is possible
                if state[pos] == state[-1][1]:
                  reward_matrix[i][pos] = reward_values[pos]
                else :
                  reward_matrix[i][pos] = reward_values['err']
                    
  return reward_matrix


In [ ]:
reward_matrix=generate_reward_matrix()
print(reward_matrix)

[[  -1.   -2.   -2.   -3.]
 [  -1.   -2.   -2.   -3.]
 [  -1.   -2.   -2.   -3.]
 ...
 [-100. -100. -100. -100.]
 [-100. -100. -100. -100.]
 [  -1.   -2.   -2.   -3.]]


Solve MDP

In [ ]:
! pip install pymdptoolbox

  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-cp37-none-any.whl size=25657 sha256=e312a3deb32a27d1d46cc8a9ce81b0413f304882fa97b35af70b7971eee94050
  Stored in directory: /root/.cache/pip/wheels/87/a9/a8/40c4e252c02e590737265742425cdd0365fafcc162441a9527
Successfully built pymdptoolbox


In [ ]:
import mdptoolbox 
mdpresultPolicy = mdptoolbox.mdp.PolicyIteration(TPMs, reward_matrix,0.999)
mdpresultValue = mdptoolbox.mdp.ValueIteration(TPMs,reward_matrix ,0.999)

In [ ]:
# Run MDP
mdpresultPolicy.run()
mdpresultValue.run()


In [ ]:
"""-------- HERE ARE THE SOLUTIONS ----------------"""

print('PolicyIteration:')
print(mdpresultPolicy.policy)
print(mdpresultPolicy.V)
print(mdpresultPolicy.iter)

print('ValueIteration:')
print(mdpresultValue.policy)
print(mdpresultValue.V)
print(mdpresultValue.iter)

PolicyIteration:
(0, 1, 1, 0, 0, 0, 0, 1, 1, 3, 0, 0, 0, 1, 1, 0, 3, 0, 0, 1, 1, 0, 0, 3, 0, 1, 1, 2, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 1, 2, 3, 0, 0, 1, 0, 2, 0, 3, 0, 3, 1, 0, 2, 0, 0, 1, 1, 3, 2, 0, 0, 2, 1, 0, 2, 0, 0, 1, 1, 0, 2, 3, 0, 1, 3, 0, 0, 2, 0, 1, 0, 3, 0, 2, 0, 1, 0, 0, 3, 2, 0, 1, 2, 0, 0, 2, 0, 2, 2, 1, 0, 0, 0, 2, 2, 1, 0, 0, 0, 2, 2, 1, 3, 0, 0, 2, 0, 1, 0, 3, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 3, 0, 3, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 1, 2, 3, 0, 0, 3, 1, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 3, 2, 0, 0, 1, 1, 0, 2, 0, 3, 2, 0, 1, 0, 0, 2, 2, 3, 1, 0, 0, 1, 2, 0, 1, 0, 0, 2, 2, 0, 1, 3, 0, 3, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 2, 1, 3, 0, 1, 0, 0, 1, 0, 0, 1, 0, 3, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 3, 0, 3, 3, 0, 1, 2, 0, 0, 0, 3, 1, 2, 0, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1, 2, 0, 2, 3, 0, 0, 1, 0, 2, 0, 3, 0, 1, 0, 2, 0, 0, 3, 1, 0, 2, 1, 0, 0, 1, 0, 0, 3, 2, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2

**Evaluation**

In [ ]:
actions={('store', 'white'): 'sw', ('store', 'blue'): 'sb', ('store', 'red'): 'sr', ('restore', 'white'): 'rw', ('restore', 'blue'): 'rb', ('restore', 'red'): 'rr'}

In [ ]:
class MDP_robot:
    def __init__(self,states, policy, warehouseSize):

        self.warehouseContent=['0'] *4
        self.color = {'white': 'w', 'blue': 'b', 'red': 'r'}
        self.states = states
        self.policy = policy
        self.ManhattanDistance=[1,2,2,3,3,4]
        self.costs = 0
        self.not_possible_actions=0
        
    def identify_state(self, warehouseContent, act): 
        #print(tuple(warehouseContent + [act]))
        for i, state in enumerate(self.states):
            if state==tuple(warehouseContent + [act]) :
                return i
        raise Exception(f'Actual warehouse content do not represent any state {tuple(warehouseContent + [act])}')

    
    def update_warehouse(self, action):
        index = self.identify_state(self.warehouseContent, actions[action])
        field = self.policy[index]
        
        #store action
        if action[0] == 'store':
            if self.warehouseContent[field] != '0':
                print(f'store is not possible at position {field} and warehouse_state {self.warehouseContent} ')
                self.not_possible_actions += 1
                return
            #update field     
            self.warehouseContent[field] = self.color[action[1]]
            #update cost function
            self.costs += self.ManhattanDistance[field]
        #restore action 
        if action[0] == 'restore':
            if self.warehouseContent[field] != self.color[action[1]]:
                print(f'restore is not possible at position {field} and warehouse_state {self.warehouseContent} ')
                self.not_possible_actions += 1
                return
                
            #update field     
            self.warehouseContent[field] = '0'
            #update cost function
            self.costs += self.ManhattanDistance[field]
        

In [ ]:
class greedy_robot:
    def __init__(self, warehouseSize):

        self.warehouseContent=[0] *warehouseSize
        self.ManhattanDistance=[1,2,2,3,3,4]
        self.color = {'white': 'w', 'blue': 'b', 'red': 'r'}
        self.costs = 0
    def update_warehouse(self, action):
        #store action 
        if action[0] == 'store':
            i = -1
            for index, pos in enumerate(self.warehouseContent):
                if pos == 0:
                    i = index
                    break
            #update field
            self.warehouseContent[i] = self.color[action[1]]
            #update cost  function 
            self.costs += self.ManhattanDistance[i]
        #restore action    
        if action[0] == 'restore':
            i = -1
            item = self.color[action[1]]
            for index, pos in enumerate(self.warehouseContent):
                if pos == item:
                    i = index
                    break
            #update field        
            self.warehouseContent[i] = 0
            #update cost function
            self.costs += self.ManhattanDistance[i]
       


In [ ]:
#import  training and test datasets 
train_data=pd.read_csv('drive/MyDrive/SAKI4/warehousetraining2x2.txt', sep='\t', header=None, names=['action', 'colour'])
order_data=pd.read_csv('drive/MyDrive/SAKI4/warehouseorder2x2.txt', sep='\t', header=None, names=['action', 'colour'])

train_actions = list(train_data.itertuples(index=False))
order_actions= list(order_data.itertuples(index=False))




Test performance of Greedy Robot

In [ ]:

Greedy_robot_train = greedy_robot(warehouseSize)
for action in train_actions:
    Greedy_robot_train.update_warehouse(action)
print(f'Training data:Number of steps of greedy robot :\t{Greedy_robot_train.costs}')

Greedy_robot_order = greedy_robot(warehouseSize)
for action in order_actions:
     Greedy_robot_order.update_warehouse(action)
print(f'Order data:Number of steps of greedy robot:\t{Greedy_robot_order.costs}')

Training data:Number of steps of greedy robot :	14401
Order data:Number of steps of greedy robot:	114


Test performance of the smart robot with value iteration 


In [ ]:
smart_robot_1_train = MDP_robot(states,mdpresultValue.policy,warehouseSize)

for action in train_actions:
    smart_robot_1_train.update_warehouse(action)
print(f'Training data:Number of steps of smart  robot  value iteration:\t{smart_robot_1_train.costs}')
print(f'number of not possible actions {smart_robot_1_train.not_possible_actions}')


store is not possible at position 1 and warehouse_state ['0', 'b', '0', 'b'] 
restore is not possible at position 0 and warehouse_state ['r', '0', '0', '0'] 
store is not possible at position 0 and warehouse_state ['r', 'r', '0', 'r'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 1 and warehouse_state ['0', 'b', '0', 'b'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 1 and warehouse_state ['0', 'b', 'w', 'b'] 
restore is not possible at position 0 and warehouse_state ['0', 'b', '0', '0'] 
store is not possible at position 1 and warehouse_state ['0', 'b', 'w', 'w'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 0 and warehouse_state ['r', 'b', '0', 'b'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 1 and warehouse_st

In [ ]:
smart_robot_1_order = MDP_robot(states,mdpresultValue.policy,warehouseSize)
for action in order_actions:
    smart_robot_1_order.update_warehouse(action)
print(f'order data:Number of steps of smart  robot  value iteration:\t{smart_robot_1_order.costs}')
print(f'number of not possible actions {smart_robot_1_order.not_possible_actions}')

store is not possible at position 1 and warehouse_state ['0', 'w', 'b', 'w'] 
order data:Number of steps of smart  robot  value iteration:	127
number of not possible actions 1


Test performance of the smart robot with policy iteration

In [ ]:
smart_robot_2_train = MDP_robot(states,mdpresultPolicy.policy ,warehouseSize)
for action in train_actions:
    smart_robot_2_train.update_warehouse(action)
print(f'Training data:Number of steps of smart  robot  policy iteration:\t{smart_robot_2_train.costs}')
print(f'number of not possible actions {smart_robot_2_train.not_possible_actions}')



store is not possible at position 1 and warehouse_state ['0', 'b', '0', 'b'] 
restore is not possible at position 0 and warehouse_state ['r', '0', '0', '0'] 
store is not possible at position 0 and warehouse_state ['r', '0', 'r', 'r'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 1 and warehouse_state ['0', 'b', '0', 'b'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 1 and warehouse_state ['0', 'b', 'w', 'b'] 
restore is not possible at position 0 and warehouse_state ['0', 'b', '0', '0'] 
store is not possible at position 1 and warehouse_state ['0', 'b', 'w', 'w'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 0 and warehouse_state ['r', 'b', '0', 'b'] 
restore is not possible at position 0 and warehouse_state ['0', '0', '0', '0'] 
store is not possible at position 1 and warehouse_st

In [ ]:
smart_robot_2_order = MDP_robot(states,mdpresultPolicy.policy,warehouseSize)
for action in order_actions:
    smart_robot_2_order.update_warehouse(action)
print(f'order data:Number of steps of smart  robot  policy iteration:\t{smart_robot_2_order.costs}')
print(f'number of not possible actions {smart_robot_2_order.not_possible_actions}')

store is not possible at position 1 and warehouse_state ['0', 'w', 'b', 'w'] 
order data:Number of steps of smart  robot  policy iteration:	127
number of not possible actions 1
